In [1]:
! pip install -U langchain_community langchain-openai langchain-anthropic langchain langgraph bs4 python-dotenv

In [2]:
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

True

In [3]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL docs
url = "https://python.langchain.com/docs/concepts/lcel/"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

### OpenAI

# Grader prompt
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a coding assistant with expertise in LCEL, LangChain expression language. \n
    Here is a full set of LCEL documentation:  \n ------- \n  {context} \n ------- \n Answer the user
    question based on the above provided documentation. Ensure any code you provide can be executed \n
    with all required imports and variables defined. Structure your answer with a description of the code solution. \n
    Then list the imports. And finally list the functioning code block. Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)


# Data model
class code(BaseModel):
    """Schema for code solutions to questions about LCEL."""

    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")


expt_llm = "gpt-4o-mini"
llm = ChatOpenAI(temperature=0, model=expt_llm)
code_gen_chain_oai = (code_gen_prompt | llm.with_structured_output(code)).with_config(
    {"run_name": "CodeGenChain", "tags": ["lcel", "codegen"]}
)
question = "How do I build a RAG chain in LCEL?"
solution = code_gen_chain_oai.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)
solution

code(prefix='To build a Retrieval Augmented Generation (RAG) chain in LCEL, you can utilize the composition primitives provided by LCEL to combine a retriever and a language model (LLM) into a single chain. The RAG chain typically involves retrieving relevant documents based on a query and then generating a response using those documents. In this example, we will create a simple RAG chain using a retriever and an LLM, and we will use the RunnableSequence to connect them.', imports='from langchain_core.runnables import RunnableSequence\nfrom langchain_core.runnables import RunnableRetriever, RunnableLLM', code='# Define the retriever and LLM\nretriever = RunnableRetriever()  # Replace with your actual retriever implementation\nllm = RunnableLLM()  # Replace with your actual LLM implementation\n\n# Create the RAG chain using RunnableSequence\nrag_chain = RunnableSequence([retriever, llm])\n\n# Invoke the RAG chain with a query\nquery = "What is the capital of France?"\nfinal_output = rag